<div align="right" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/Logo blue_dark.png"  style="width:25px" align="right";/>
</div>

# Optimising SQL queries
© ExploreAI Academy

In this notebook, we dive into the different ways we can optimise our SQL queries to make them more efficient.

## Learning objectives

By the end of this train, you should be able to write more efficient SQL queries by:
- Using column names instead of `*`.
- Creating JOINS with `INNER` instead of `WHERE`.
- Avoiding using the wildcard `%` at the beginning of a predicate.
- Avoiding using functions when searching for patterns.
- Avoiding using calculated fields in the `JOIN` and `WHERE` clauses.
    

## Connecting to our MySQL database

In this train, we'll be using the US Flights database with actual US flights data to illustrate the optimisation techniques with large amounts of data. To follow along, ensure that you have downloaded the database file, `flights.db`. 

First, let's load our sample database:

In [2]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.
%load_ext sql


In [3]:
# Load the Flights database stored in your local machine. 
# Make sure the file is saved in the same folder as this notebook.
%sql sqlite:///flights.db 

Here is a view of all of our tables in the database:

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/flights_db_ER.png"  style="width:500px";/>
<br>
<br>
    <em>Figure 1: Flights ERD</em>
</div>

The Flights database consists of the following tables:

- **flights**:     all domestic flights in the USA in 2008
- **carriers**:     lookup table for all the carriers
- **airports**:     lookup table for all the airports
- **planes**:     lookup table for the planes

## Why do we need to optimise our queries?

SQL queries might **take a long time to run** depending on the size of the database and the nature of the query. In addition, writing large SQL queries has the **potential to jam a server** (in a server-based configuration). For example, if a server spends too much time on a single query, other queries need to "wait in line" for the server to complete processing the one query. As a result, it is critical that we write more efficient queries, for example, queries that only read the data that is required.

In this notebook, we will use iPython's built-in `%%time` magic command which will measure the execution time of code within a given Jupyter cell.

**Note:** The SQL queries and databases we used in this notebook are not very complex. For this reason, we might not see large differences in speed between inefficient and efficient SQL queries. Sometimes we might even witness “efficient” queries being slower than “inefficient” queries. This is because other factors outside of the database such as computer performance or network connection can impact the amount of time it takes to complete a query.

However, writing efficient queries even when databases are simple will ensure that your queries scale as your database grows in size and complexity. Well-written SQL queries are good coding practice and will always retrieve data quickly. 



## Common Table Expressions (CTEs)

CTEs simplify difficult queries by breaking them down into smaller, more understandable components and temporarily storing the results for later reference by other queries.  While CTEs make the code more readable and reusable, there are many other ways to optimise our queries to make them more efficient.


Here are a few other ways we can optimise our queries:

## Use column names instead of `*`

Instead of using  `*` in our select statements, which selects all the columns in the table, we should rather use the specific column names we are interested in.
For example, say we are only interested in the tail number and delay before departure for the first 50 flights. 
 
### Inefficient query

In [3]:
%%time
%%sql

SELECT * 
FROM flights

 * sqlite:///flights.db
Done.
CPU times: total: 3.27 s
Wall time: 5.55 s


### Efficient query

In [4]:
%%time
%%sql

SELECT TailNum, DepDelay 
FROM flights 
LIMIT 50

 * sqlite:///flights.db
Done.
CPU times: total: 15.6 ms
Wall time: 50.2 ms


TailNum,DepDelay
N712SW,8.0
N772SW,19.0
N428WN,8.0
N612SW,-4.0
N464WN,34.0
N726SW,25.0
N763SW,67.0
N428WN,-1.0
N689SW,2.0
N648SW,0.0


Many SQL developers use `SELECT * FROM table` to get a look at column data, which becomes taxing if there are many records (normally a few million or more).

Returning only the required columns decreases the overall amount of data read and, as a result, the runtime. For the same reasons, it is also vital to use `LIMIT` to limit the number of rows in the result to only the required rows.

## Create `JOINS` with `INNER`
When extracting data from two databases and joining them on specific columns, we could use the `WHERE` clause to join the tables on those columns in most cases. Assume we want to add the carrier's description to the Flights database. That will require querying the carriers table and the flights table for codes that match the unique carrier name in the flights table.
 
### Inefficient query

In [ ]:
%%time
%%sql

SELECT FlightNum, TailNum, Description
FROM flights, carriers
WHERE carriers.Code = flights.UniqueCarrier
LIMIT 2500

### Efficient query

In [ ]:
%%time
%%sql

SELECT FlightNum, TailNum, Description
FROM flights
INNER JOIN carriers
ON carriers.Code = flights.UniqueCarrier
LIMIT 2500;

Joining tables using the `WHERE` clause is similar to doing a `CROSS` join. This type of join will first generate all possible combinations of the values in the joining columns and then filter these results according to the specified condition. This means we iterate through the combination of records from both tables. However, when an inner join operation is performed, only records in the left table are iterated through to find matches in the right table. 

## Avoid using the wildcard (`%`) at the beginning of a `LIKE` operator

When every row in a table has an index, it is possible to search the table using the index, making it easier and faster to find the values we are looking for. Whether we search using the `%` wildcard at the beginning or the end of the `LIKE` operator also affects how efficient our query is. 

Suppose we want the tail number and distance of all flights whose tail number starts with `N7` 
 
### Inefficient query

In [7]:
%%time
%%sql

SELECT TailNum, Distance
FROM flights
WHERE TailNum LIKE '%N7%'
LIMIT 2500;

 * sqlite:///flights.db
Done.
CPU times: total: 31.2 ms
Wall time: 302 ms


### Efficient query

In [8]:
%%time
%%sql

SELECT TailNum, Distance
FROM flights
WHERE TailNum LIKE 'N7%'
LIMIT 2500;

 * sqlite:///flights.db
Done.
CPU times: total: 0 ns
Wall time: 7.38 ms



Since it does not use the index to aid in the search, using a wildcard at the beginning of a pattern necessitates a full table scan. We should rather utilise a trailing wildcard (`%` at the end of the operator) to search for text at the beginning of a field. This will take advantage of any existing indexing.

## Avoid using functions when searching for patterns
Let's say we also wanted to return the origin and departure delay of all flights from a specific origin. 
 
### Inefficient query

In [9]:
%%time
%%sql

SELECT Origin, DepDelay 
FROM flights
WHERE LOWER(Origin) = 'las'
LIMIT 7000;

 * sqlite:///flights.db
Done.
CPU times: total: 31.2 ms
Wall time: 366 ms


### Efficient query

In [10]:
%%time
%%sql

WITH CTE AS
(SELECT LOWER(Origin) Origin_lower, DepDelay FROM flights)

SELECT * FROM CTE WHERE Origin_lower = 'las'
LIMIT 7000;

 * sqlite:///flights.db
Done.
CPU times: total: 31.2 ms
Wall time: 69.8 ms


If we use functions when searching for patterns, the index-based optimisations can’t be used. 
One alternative method is to use CTEs instead.

## Avoid using calculated fields in the JOIN and WHERE clauses

Suppose we want to return the flights where the departure delay was more than 20% of the flight time. We would need to calculate the value of 20% of the `AirTime` column and then compare that to the `DepDelay` column for each row.
 
### Inefficient query

In [11]:
%%time
%%sql

SELECT TailNum, DepDelay
FROM flights
WHERE AirTime/5 < DepDelay
LIMIT 5000;

 * sqlite:///flights.db
Done.
CPU times: total: 0 ns
Wall time: 16.9 ms


### Efficient query

In [12]:
%%time
%%sql

WITH CTE AS 
(SELECT AirTime/5 as fifth_time, DepDelay, TailNum FROM flights)

SELECT TailNum, DepDelay 
FROM CTE
WHERE CTE.fifth_time < CTE.DepDelay
LIMIT 5000;

 * sqlite:///flights.db
Done.
CPU times: total: 0 ns
Wall time: 9.07 ms


Calculated fields in `JOIN` and `WHERE` clauses also prevent indices from being used. CTEs can be used instead as well.

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>